In [34]:
require 'nn';

### Prepair data

In [35]:
local matio = require 'matio'
data = matio.load('ex4data1.mat')

In [36]:
trainset = {}
trainset.data = data.X
trainset.label = data.y[{ {}, 1}]

In [45]:
trainset

{
  data : DoubleTensor - size: 5000x400
  size : function: 0x40ebcb00
  label : DoubleTensor - size: 5000
}


In [41]:
setmetatable(trainset,
    {__index = function(t,i)
                return {t.data[i], t.label[i]}
        end}
);

function trainset:size()
    return self.data:size(1)
end

In [46]:
mean = {}
stdv = {}
for i=1,400 do
    mean[i] = trainset.data[{ {},{i} }]:mean()
    stdv[i] = trainset.data[{ {}, {i} }]:std()
    --print(i .. 'th mean: ' .. mean[i])
    --print(i .. 'th std dev: ' .. stdv[i])
    trainset.data[{ {},{i} }]:add(-mean[i])
    if stdv[i] ~= 0 then
        trainset.data[{ {},{i} }]:div(stdv[i])
    end
end

### define model

In [47]:
net = nn.Sequential()
net:add(nn.Linear(400,25))
net:add(nn.Sigmoid())
net:add(nn.Linear(25,10))
net:add(nn.Sigmoid())
net:add(nn.LogSoftMax())

### define loss function

In [48]:
criterion = nn.ClassNLLCriterion()

### let's train

In [49]:
trainer = nn.StochasticGradient(net, criterion)
trainer.learningRate = 0.001
trainer.maxIteration = 1e1

In [50]:
trainer:train(trainset)

# StochasticGradient: training	


# current error = 2.2922343367451	


# current error = 2.2630574409607	


# current error = 2.2332608902148	


# current error = 2.2021899982022	


...

# current error = 2.1377635123074	


# current error = 2.1059568652053	


# current error = 2.0753091127596	


# current error = 2.0462203006493	


# current error = 2.0189019314781	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 2.0189019314781	



### test

In [ ]:
correction = 0
for i=1,trainset:size() do
    local answer = trainser.label[i]
    local prediction = net:forward(trainset.data[i])
    local confidences, indices = torch.sort(prediction, true)